In [135]:
##Imported required Python libraries 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pickle

In [136]:
#reading train dataset from the given csv and drooping NA values
df_train = pd.read_csv ('D:\INTERVIEW_DS_ML\dataset_train_1.csv')
df_train_final = df_train.dropna()

In [137]:
#Applied label encoding variable for "yes"/ "no" and for target variables
enc_train = LabelEncoder()
df_train_final['Var5']=enc_train.fit_transform(df_train_final.Var5.values)
df_train_final['Var6']=enc_train.fit_transform(df_train_final.Var6.values)
df_train_final['Target']=enc_train.fit_transform(df_train_final.Target.values)
df_train_final.head()

<ipython-input-137-24eabef6a9c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final['Var5']=enc_train.fit_transform(df_train_final.Var5.values)
<ipython-input-137-24eabef6a9c5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final['Var6']=enc_train.fit_transform(df_train_final.Var6.values)
<ipython-input-137-24eabef6a9c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,Unnamed: 0,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Target
0,1,K1,128.0,415.0,01dc820b1aa6175cef997a09f88da685,0,1,25.0,265.1,110.0,...,99.0,16.78,244.7,91.0,11.01,10.0,3.0,2.70,1.0,0
2,3,N5,137.0,415.0,8b5d5b1825d00c99b28cd8464215739b,0,0,0.0,243.4,114.0,...,110.0,10.30,162.6,104.0,7.32,12.2,5.0,3.29,0.0,0
4,5,O2,75.0,415.0,ffd5bfdb4243cbe84b2e1fbabaac2d2a,1,0,0.0,166.7,113.0,...,122.0,12.61,186.9,121.0,8.41,10.1,3.0,2.73,3.0,0
5,6,A2,118.0,510.0,25065790aca03b4476e8c50ad9dbbec6,1,0,0.0,223.4,98.0,...,101.0,18.75,203.9,118.0,9.18,6.3,6.0,1.70,0.0,0
7,8,M6,147.0,415.0,52b28dc642bfb9ab765d4df0e7553c67,1,0,0.0,157.0,79.0,...,94.0,8.76,211.8,96.0,9.53,7.1,6.0,1.92,0.0,0


In [138]:
#Defining independent and dependent variables from the data set
independent_columns=list(df_train_final.columns)
independent_columns.remove('Target')
independent_columns.remove('Var1')
independent_columns.remove('Var4')
dependent_columns=['Target']

In [139]:
#Feature selection using chisquare and find p values 
X_train,X_test,y_train,y_test=train_test_split(df_train_final[independent_columns],
                                              df_train_final[dependent_columns],test_size=0.2,random_state=100)
ftr_p_value=chi2(X_train,y_train)

In [140]:
#Chi square continuation for feature selction by choosing p values from array
p_values=pd.Series(ftr_p_value[1])
p_values.index=X_train.columns
final_features=p_values.sort_index(ascending=False).head(10)
features_df=pd.DataFrame(final_features,columns=['Var'])

In [141]:
#filtered final columns post feature selection 
list_f_column=list(features_df.index)
list_f_column

['Var9',
 'Var8',
 'Var7',
 'Var6',
 'Var5',
 'Var3',
 'Var20',
 'Var2',
 'Var19',
 'Var18']

In [142]:
#Train and test again using feature selected columns
X_train,X_test,y_train,y_test=train_test_split(df_train_final[list_f_column],
                                              df_train_final[dependent_columns],test_size=0.2,random_state=100)

In [143]:
# fit model with training data by applying XGboost Classifier algo
crdts_model = XGBClassifier()
crdts_model.fit(X_train, y_train)

[14:34:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\BITS_PILANI\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\BITS_PILANI\Anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [144]:
# make predictions for test data
y_pred = crdts_model.predict(X_test)
crdts_predictions = [round(value) for value in y_pred]

In [145]:
# evaluate predictions
crdts_accuracy = accuracy_score(y_test, crdts_predictions)
print("Credit Suisse model Accuracy: %.2f%%" % (accuracy * 100.0))

Credit Suisse model Accuracy: 91.84%


In [146]:
# save the model to disk using pickle
crdts_filename = 'D:\INTERVIEW_DS_ML\BNCModel.sav'
pickle.dump(model, open(crdts_filename, 'wb'))

In [147]:
# load the model from disk
crdts_loaded_model = pickle.load(open(crdts_filename, 'rb'))


In [148]:
#reading test data from the given csv and drop NA values as well and encode relevant values
df_testdata = pd.read_csv ('D:\INTERVIEW_DS_ML\dataset_test_1.csv')
df_test_final = df_testdata.dropna()
df_test_final['Var5']=enc_train.fit_transform(df_test_final.Var5.values)
df_test_final['Var6']=enc_train.fit_transform(df_test_final.Var6.values)
df_test_final['Target']=enc_train.fit_transform(df_test_final.Target.values)

<ipython-input-148-9ef94614a493>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_final['Var5']=enc_train.fit_transform(df_test_final.Var5.values)
<ipython-input-148-9ef94614a493>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_final['Var6']=enc_train.fit_transform(df_test_final.Var6.values)
<ipython-input-148-9ef94614a493>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [149]:
#Apply feature selected columns in test data 
df_test_final_X=df_test_final[list_f_column]

In [150]:
y_final_pred = crdts_model.predict(df_test_final_X)

In [151]:
print(y_final_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0]


In [152]:
#Assign predicted values and traget to the test data column and decode the same
df_test_final['Predicted Value']=y_final_pred
df_test_final['Predicted Value']=np.where(df_test_final['Predicted Value']==0,'false','true')
df_test_final['Target']=np.where(df_test_final['Target']==0,'false','true')
df_test_final.head()

<ipython-input-152-a1ab592b47d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_final['Predicted Value']=y_final_pred
<ipython-input-152-a1ab592b47d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_final['Predicted Value']=np.where(df_test_final['Predicted Value']==0,'false','true')
<ipython-input-152-a1ab592b47d3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,Unnamed: 0,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Target,Predicted Value
0,1,C2,71.0,415.0,0731015efe10b09a50d71bb517c18920,0,0,0.0,211.2,70.0,...,21.48,225.8,104.0,10.16,12.3,3.0,3.32,0.0,false,false
2,3,N5,87.0,510.0,41e0f4c5ca086d6e7eb5cfed9d74c4d0,0,0,0.0,223.2,109.0,...,10.84,289.3,83.0,13.02,14.5,4.0,3.92,3.0,false,false
5,6,W3,95.0,415.0,6344244fd70aa4433b6ddf8c966b7041,0,0,0.0,134.4,104.0,...,12.95,236.5,80.0,10.64,9.4,3.0,2.54,1.0,false,false
7,8,O1,66.0,415.0,8e9ec27462e9ddaeedb264e5933b8713,0,0,0.0,170.5,103.0,...,21.62,197.3,138.0,8.88,10.5,2.0,2.84,2.0,false,false
9,10,M2,204.0,510.0,fdbe535b7c3852be7f8f635235c6194a,0,0,0.0,205.2,145.0,...,13.16,191.4,77.0,8.61,14.1,5.0,3.81,3.0,false,false


In [153]:
#Save the final results to a CSV 
df_test_final.to_csv(r'D:\INTERVIEW_DS_ML\BNCFinalResults.csv')